In [110]:
import pandas as pd

csv = pd.read_csv('index_label_to_category.csv')
# csv.set_index('landmark_id')
csv


,landmark_id,category
0,0,http://commons.wikimedia.org/wiki/Category:Sea...
1,1,http://commons.wikimedia.org/wiki/Category:Fra...
2,2,http://commons.wikimedia.org/wiki/Category:Rin...
3,3,http://commons.wikimedia.org/wiki/Category:Tak...
4,4,http://commons.wikimedia.org/wiki/Category:Iep...
...,...,...
101297,101297,http://commons.wikimedia.org/wiki/Category:Bor...
101298,101298,http://commons.wikimedia.org/wiki/Category:Hol...
101299,101299,http://commons.wikimedia.org/wiki/Category:Eri...
101300,101300,http://commons.wikimedia.org/wiki/Category:End...


In [111]:
country_map = pd.read_csv('countries_codes_and_coordinates.csv')
country_map


,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""
...,...,...,...,...,...,...
251,Wallis and Futuna,"""WF""","""WLF""","""876""","""-13.3""","""-176.2"""
252,Western Sahara,"""EH""","""ESH""","""732""","""24.5""","""-13"""
253,Yemen,"""YE""","""YEM""","""887""","""15""","""48"""
254,Zambia,"""ZM""","""ZMB""","""894""","""-15""","""30"""


In [112]:
#test for country code
# url = 'https://commons.wikimedia.org/wiki/Category:Seal_Bay_Conservation_Park'
def countryCodeFromURL (url):
    dfs = pd.read_html(url)
    country = dfs[0][1][4]
    country = country.split(',')[-1][1:]
    twoCode = country_map.loc[country_map['Country'] == country]
    twoCode = twoCode['Alpha-2 code'].tolist()[0]
    twoCode = twoCode[2:-1]
    return twoCode
# twoCode

In [120]:
url = 'https://commons.wikimedia.org/wiki/Category:Takeda-jinja'
dfs = pd.read_html(url)
country = dfs[0][1][4]
print(country)
country = country.split(',')[-1][1:]
print(country)
twoCode = country_map.loc[country_map['Country'] == country]
twoCode = twoCode['Alpha-2 code'].tolist()[0]
twoCode = twoCode[2:-1]
twoCode

Takeda Shingen
akeda Shingen


IndexError: list index out of range

In [114]:
from bs4 import BeautifulSoup
import requests

def nameFromURL (soup):
    landmark_name = soup.find(id = 'wdinfoboxcaption')
    landmark_name = landmark_name.text
    return landmark_name
# print(landmark_name)

In [115]:
def latLongFromURL (soup):
    allLinks = soup.find(id="bodyContent").find_all("a")
    linkToScrape = 0

    for link in allLinks:
        if 'geohack' in link['href']: 
            linkToScrape = 'https:' + link['href']

    latLongDFs = pd.read_html(linkToScrape)
    latLong = latLongDFs[0][1][0].split('E')[1]
    return latLong
# print(latLong)

In [116]:
def imageFromURL (soup):
    image = soup.find('img')['src']
    suffix = image.split('/')[-1][3:]
    suffix = '500' + suffix
    prefix = image[:image.rindex('/')+1]
    image = prefix + suffix
    return image

In [117]:
pd.options.mode.chained_assignment = None  # default='warn'
csv['landmark_name'] = 0
csv['image_url'] = 0
csv['country_code'] = 0
csv['lat_long'] = 0
for (index, row) in csv.iterrows():
    url = row['category']
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    if index == 5:
        break
    try:
        csv['landmark_name'][index] = nameFromURL(soup)
    except:
        print(row['category'])
        csv['landmark_name'][index] = None
    try:
        csv['image_url'][index] = imageFromURL(soup)
    except:
        print(row['category'])
        csv['image_url'][index] = None
    try:
        csv['country_code'][index] = countryCodeFromURL(url)
    except:
        print(row['category'])
        csv['country_code'][index] = None
    try:
        csv['lat_long'][index] = latLongFromURL(soup)
    except:
        print(row['category'])
        csv['lat_long'][index] = None

csv.to_csv('landmarks.csv')
csv

AUS
http://commons.wikimedia.org/wiki/Category:Seal_Bay_Conservation_Park
http://commons.wikimedia.org/wiki/Category:Frances_M._MacKay_House
http://commons.wikimedia.org/wiki/Category:Frances_M._MacKay_House
Sweden
akeda Shingen
http://commons.wikimedia.org/wiki/Category:Takeda-jinja
elgium
http://commons.wikimedia.org/wiki/Category:Ieperlee


,landmark_id,category,landmark_name,image_url,country_code,lat_long
0,0,http://commons.wikimedia.org/wiki/Category:Sea...,Seal Bay Conservation Park,https://upload.wikimedia.org/wikipedia/commons...,NaN,"-35.9986, 137.363"
1,1,http://commons.wikimedia.org/wiki/Category:Fra...,Building at 10 Follen Street,https://upload.wikimedia.org/wikipedia/commons...,NaN,None
2,2,http://commons.wikimedia.org/wiki/Category:Rin...,Ringsjön,https://upload.wikimedia.org/wikipedia/commons...,SE,"55.8722, 13.5186"
3,3,http://commons.wikimedia.org/wiki/Category:Tak...,Takeda Shrine,https://upload.wikimedia.org/wikipedia/commons...,None,"35.6867, 138.5775"
4,4,http://commons.wikimedia.org/wiki/Category:Iep...,Ieperlee,https://upload.wikimedia.org/wikipedia/commons...,None,"51, 2.8"
...,...,...,...,...,...,...
101297,101297,http://commons.wikimedia.org/wiki/Category:Bor...,0,0,0,0
101298,101298,http://commons.wikimedia.org/wiki/Category:Hol...,0,0,0,0
101299,101299,http://commons.wikimedia.org/wiki/Category:Eri...,0,0,0,0
101300,101300,http://commons.wikimedia.org/wiki/Category:End...,0,0,0,0


In [118]:
csv[]

SyntaxError: invalid syntax (<ipython-input-118-c7593165d623>, line 1)